<a href="https://colab.research.google.com/github/pullz6/Research_inspired_Self-Explaining-Neural-Networks-for-Business-Process-Monitoring/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [64]:
#Importing libraries
import pandas as pd
import numpy as np
from graphviz import Digraph
from matplotlib import pyplot as plt

In [65]:
df = pd.read_csv("Helpdesk.csv")

In [67]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
4,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21348 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21348 non-null  object
 1   lifecycle:transition  21348 non-null  object
 2   org:resource          21348 non-null  object
 3   time:timestamp        21348 non-null  object
 4   Activity              21348 non-null  object
 5   Resource              21348 non-null  object
 6   case:concept:name     21348 non-null  object
 7   case:variant          21348 non-null  object
 8   case:variant-index    21348 non-null  int64 
 9   case:creator          21348 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.6+ MB


In [68]:
df.drop_duplicates(inplace=True)

In [69]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], format='ISO8601')

<ipython-input-69-97bac535b9d0>:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], format='ISO8601')


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21221 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21221 non-null  object
 1   lifecycle:transition  21221 non-null  object
 2   org:resource          21221 non-null  object
 3   time:timestamp        21221 non-null  object
 4   Activity              21221 non-null  object
 5   Resource              21221 non-null  object
 6   case:concept:name     21221 non-null  object
 7   case:variant          21221 non-null  object
 8   case:variant-index    21221 non-null  int64 
 9   case:creator          21221 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.8+ MB


In [71]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
5,Assign seriousness,complete,Value 2,2010-01-13 12:26:04+02:00,Assign seriousness,Value 2,Case2748,Variant 1,1,Fluxicon Disco


In [72]:
df['new_date'] = [d.date() for d in df['time:timestamp']]
df['new_time'] = [d.time() for d in df['time:timestamp']]

In [73]:
df['concept:name'].unique()

array(['Assign seriousness', 'Take in charge ticket', 'Resolve ticket',
       'Closed', 'Wait', 'Create SW anomaly', 'Insert ticket',
       'Schedule intervention', 'RESOLVED', 'INVALID', 'VERIFIED',
       'Resolve SW anomaly', 'Require upgrade', 'DUPLICATE'], dtype=object)

In [74]:
#Checking if we can concept:name and activity is the same as well as the org:resource and the resource, so that we can delete this feild.
df['Checker_1'] = (df['concept:name'] == df['Activity'])
df['Checker_2'] = (df['org:resource'] == df['Resource'])

In [75]:
#Let's see if it all the same, if True.
df['Checker_2'].unique()

array([ True])

In [76]:
#Lets drop it since it is the same
df.drop(['concept:name','org:resource','Checker_1','Checker_2','case:creator','time:timestamp'],axis=1, inplace=True)

In [77]:
#Lets create a stage number for each activity stage of the help desk, this will help us easily identify where the case is in terms of the process

#This is also to immulate the label encoder
#First create a dataframe with stage numbers and the stage names
stage = pd.DataFrame()
stage['Stage_no'] = [1,2,3,4,5,6,7,8,9,9.1,9.2,10]
stage['Activity'] = ['Insert ticket','Assign seriousness','Take in charge ticket','Wait','Require upgrade','Schedule intervention','Resolve ticket','VERIFIED','RESOLVED','INVALID', 'DUPLICATE','Closed']

In [78]:
df=pd.merge(df,stage,on='Activity', how='left')

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21221 entries, 0 to 21220
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   lifecycle:transition  21221 non-null  object 
 1   Activity              21221 non-null  object 
 2   Resource              21221 non-null  object 
 3   case:concept:name     21221 non-null  object 
 4   case:variant          21221 non-null  object 
 5   case:variant-index    21221 non-null  int64  
 6   new_date              21221 non-null  object 
 7   new_time              21221 non-null  object 
 8   Stage_no              21142 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 1.5+ MB


In [80]:
df.head()

,lifecycle:transition,Activity,Resource,case:concept:name,case:variant,case:variant-index,new_date,new_time,Stage_no
0,complete,Assign seriousness,Value 2,Case3608,Variant 33,33,2010-01-13,08:40:25,2.0
1,complete,Take in charge ticket,Value 2,Case3608,Variant 33,33,2010-01-29,08:52:27,3.0
2,complete,Resolve ticket,Value 2,Case3608,Variant 33,33,2010-01-29,08:52:34,7.0
3,complete,Closed,Value 5,Case3608,Variant 33,33,2010-02-13,08:52:48,10.0
4,complete,Assign seriousness,Value 2,Case2748,Variant 1,1,2010-01-13,12:26:04,2.0


One-hot-Encoding

In [81]:
from sklearn.preprocessing import OneHotEncoder

In [82]:
encoder = OneHotEncoder(sparse_output=False)

In [83]:
activity = df['Activity'].tolist()
one_hot_encoded = encoder.fit_transform(activity)

ValueError: Expected 2D array, got 1D array instead:
array=['Assign seriousness' 'Take in charge ticket' 'Resolve ticket' ...
 'Resolve ticket' 'Resolve ticket' 'Closed'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

Model building

In [84]:
!pip install keras

In [86]:
#Creating the model architecture

In [85]:
#Importing the required libriares since the LSTM model will be done with Pytorch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.utils import get_file
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import BatchNormalization